In [326]:
# This helper class is a good-faith effort to limit the volume of code required to store, process, 
# and print (to the screen) various ephemera related to and verily defining the irrigation events
# with which we are concerned fortwith. Tragically, no analogous simplifying class exists
# with which the volume of supporting comment could have been limited. The time you've spend reading this?
# You're never going to get it back. Carry on.

class irrEvent:
    # These are documented in notes from 5/1/21. Units: mm/min
    rate1 = 19.05/60 # Backyard
    rate2 = 7.143/60 # South side yard
    rate3 = 2.381/60 # Front yard

    def __init__(self, target, freq):
        self.target = float(target) # Target precipitation per week
        self.freq = float(freq) # Irrigation frequency, e.g. how many days per week the sprinklers should run
        self.event = self.target/self.freq # How much irrigation is necessary per event
        
        # Uses the init value to calculate how long to irrigate for each zone
        self.zone1 = round(self.event/self.rate1) 
        self.zone2 = round(self.event/self.rate2)
        self.zone3 = round(self.event/self.rate3)
        
    def showTimes(self):
        print("Zone 1:", self.zone1, 'minutes')
        print("Zone 2:", self.zone2, 'minutes')
        print("Zone 3:", self.zone3, 'minutes')

In [353]:
# Get historical data from meteostat
import config
from datetime import datetime, date, timedelta, time
from meteostat import Point, Daily, Stations
from requests.auth import HTTPBasicAuth
import json

def historicalRainfall(numOfDays, location):
    # Set time period
    today = datetime.combine(date.today(), time()) 
    hist = today - timedelta(days=numOfDays)
    stations = Stations()
    station = stations.id('icao', location).fetch()
    # print(station)
    data = Daily(station, start=hist, end=now)
    data = data.normalize()
    #data = data.aggregate('1W')
    data = data.fetch()
    result = data['prcp'].sum()
    # print result
    return result


# Two functions for getting forecast data
# API call to get the data from openweathermap.org
def getForecast():
    key = config.OPENWX_CONFIG['key']
    resource = 'http://api.openweathermap.org/data/2.5/forecast?'
    zcode = config.OPENWX_CONFIG['zipcode']
    units = 'units=metric&' # Standard, metric and imperial units are available. Standard units are default
    # mode = 'mode=xml&' #Response format. JSON format is used by default. To get data in XML format use mode=xml
    # cnt = 'cnt=numForecasts #Number of timestamps, which will be returned in the API response
    # lang = someLang #Get the output in your language
    
    url = resource + zcode + units + key

    response = requests.request(
        "GET",
        url
    )
    payload = json.loads(response.text)
    return(payload)

# Process the response
def forecast():
    payload = getForecast()
    
    today = datetime.combine(date.today(), time()) + timedelta(days=1)
    tomorrow = today + timedelta(days=2)

    rain2day = 0
    rain2mrw = 0

    for forecast in payload['list']:
        dt = forecast['dt']
        timestamp = datetime.fromtimestamp(dt)
        timestamp = timestamp - timedelta(hours=6)

        if (forecast['weather'][0]['main'] == 'Rain' 
            and timestamp > today 
            and timestamp < tomorrow):
#             print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
#             print("it's going to rain ", forecast['rain']['3h'], "mm")
            rain2day += forecast['rain']['3h']
        if (forecast['weather'][0]['main'] == 'Rain' 
            and timestamp >= (tomorrow) 
            and timestamp < (tomorrow + timedelta(days=1))):
#             print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
#             print("it's going to rain ", forecast['rain']['3h'], "mm")
            rain2mrw += forecast['rain']['3h']

#         print("it's going to rain ", round(rain2day), "mm today")
#         print("it's going to rain ", round(rain2mrw), "mm tomorrow")
    results = {
        'today': rain2day,
        'tomorrow': rain2mrw
    }
    return results
                


In [354]:
%%html
<style>
table {align:left;display:block}
</style>

### Irrigation Data
#### Irrigation Rates (Per Hour)
|        | Inches | mm    |
|--------|--------|-------|
| Zone 1 | 3/4    | 19.05 |
| Zone 2 | 9/32   | 7.143 |
| Zone 3 | 3/32   | 2.381 |

#### Irrigation Target (Weekly)
|           | Inches | mm    |
|-----------|--------|-------|
| New Grass | 2"     | 50.8  |
|Established| 1"     | 25.4  |

In [355]:
# How long to irrigate 2x a day, every day
newGrass = irrEvent(50.8, 14)
print("\nNew Grass Irrigation Times")
newGrass.showTimes()

# How much to irrigate 1x per day
medAmt = (50.8+25.4)/2
phase1 = irrEvent(medAmt, 7)
print("\nPhase 1 Irrigation Times")
phase1.showTimes()

# How much to irrigate 4x per week
phase2 = irrEvent(25.4, 4)
print("\nPhase 2 Irrigation Times")
phase2.showTimes()

# How much to irrigate 2x per week
established = irrEvent(25.4, 2)
print("\nEstablished Grass Irrigation Times")
established.showTimes()

goal = {
    'newGrass': newGrass,
    'phase1': phase1,
    'phase2': phase2,
    'established': established
}


New Grass Irrigation Times
Zone 1: 11 minutes
Zone 2: 30 minutes
Zone 3: 91 minutes

Phase 1 Irrigation Times
Zone 1: 17 minutes
Zone 2: 46 minutes
Zone 3: 137 minutes

Phase 2 Irrigation Times
Zone 1: 20 minutes
Zone 2: 53 minutes
Zone 3: 160 minutes

Established Grass Irrigation Times
Zone 1: 40 minutes
Zone 2: 107 minutes
Zone 3: 320 minutes


In [356]:
# Get rainfall accumulation for last 6 days (7dayHist)
last6Days = historicalRainfall(numOfDays=6, location='KCMI')

# Get 7 day irrigation total (irrHist)
irrHistory = 0

# Get 7 day irrigation total (irrHist)
precip = last6Days + irrHistory

# Get watering gap (7day - target = gap). Options are newGrass, phase1, phase2, and established
gap = precip-goal['newGrass'].target

# Get forecasts & calculate forward look
forecast = forecast()
predict = forecast['today'] + 0.5*forecast['tomorrow']

In [357]:
# Today's Conditions
print("7 Day Rainfall & Irrigation: ", round(precip))
print("24hr Forecast: ", forecast['today'])
print("Weekly Accumulation Target: ", round(goal['newGrass'].target))
print("Today's Water shortgage:", round(gap + forecast['today']))

7 Day Rainfall & Irrigation:  23.0
24hr Forecast:  2.89
Weekly Accumulation Target:  51
Today's Water shortgage: -25.0


In [358]:
eventResult = precip+forecast['today']+goal['newGrass'].event
if (precip+predict+goal['newGrass'].event < goal['newGrass'].target):
    print()
    print("Recommend Run Irrigation")
    print("Proposed Correction: ",round(goal['newGrass'].event))
    print("Corrected Accumulation: ", precip+forecast['today']+goal['newGrass'].event)
    print("Corrected Gap: ", gap + forecast['today']+goal['newGrass'].event)
else:
    print("Delay 1 day")


Recommend Run Irrigation
Proposed Correction:  4
Corrected Accumulation:  29.918571428571433
Corrected Gap:  -20.881428571428565
